In [40]:
import pandas as pd
import plotly.graph_objects as go

In [153]:
# data from: https://www.singstat.gov.sg/publications/population/marriages-and-divorces
# Using sheet T1: T1 Cumulative Proportion of Dissolved Resident Marriages

# inspired from: Straits Time Article titled:
# Largest share of divorces and annulments in S’pore is among those wed for between 5 and 10 years
# https://www.straitstimes.com/singapore/largest-share-of-divorces-and-annulments-in-s-pore-is-among-those-wed-for-between-5-and-10-years

file_path = "data_files/Dissolution of Marriages Among Marriage Cohorts.xlsx"
df = pd.read_excel(file_path, sheet_name="T1", skiprows=6)# index_col=0) #x-axis: Before xth Anniversary, y-axis: Marriage Cohort
df = df.rename({"Unnamed: 0": "marriage_cohort"}, axis=1)
df = df.rename(columns={col: f"before_{col}_anniversary" for col in df.columns if col != "marriage_cohort"})
df['before_0_anniversary'] = 0 #adding 0th year for calculations

columns_order = ['marriage_cohort', 'before_0_anniversary'] + [col for col in df.columns[1:-1]]
df = df[columns_order]

# Filter columns to plot only "before_{num}_anniversary" where num is in multiples of 5 and "before_1_anniversary"
columns_to_plot = [column for column in df.columns if column.startswith("before_") and (int(column.split("_")[1]) % 5 == 0)]

# Calculate incremental values for each selected column
df_incremental = df[columns_to_plot].set_index(df["marriage_cohort"]).diff(axis=1).reset_index().round(2)

In [170]:
def snake_case_to_readable(text):
    """
    Convert a snake_case string to a more readable format.
    Example: "before_1_anniversary" will be converted to "Before 1 Anniversary".
    """
    words = text.split('_')
    capitalized_words = [word.capitalize() for word in words]
    return ' '.join(capitalized_words)

In [169]:
# Create a figure
fig = go.Figure()

# Add trace for each "before_{num}_anniversary" column
for column in df.columns[1:]:
    fig.add_trace(go.Scatter(x=df.marriage_cohort, y=df[column], mode='lines', name=column))

# Update layout
fig.update_layout(
    title="Cumulative Graph of before_{num}_anniversary",
    xaxis_title="Marriage Cohort",
    yaxis_title="Cumulative Value",
    showlegend=True,
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
    hovermode="x"
)

# Show the plot
fig.show()


In [172]:
# Create the column_names_mapping dictionary using dictionary comprehension
column_names_mapping = {col: snake_case_to_readable(col) for col in columns_to_plot[1:]}

# Create a figure
fig = go.Figure()

# Add trace for each selected column as a stacked cumulative bar chart
for column in columns_to_plot[1:]:
    if column != "marriage_cohort":
        fig.add_trace(go.Bar(
            x=df_incremental["marriage_cohort"],
            y=df_incremental[column],
            name=column_names_mapping[column],  # Use the mapped name from the dictionary
        ))

# Update layout
fig.update_layout(
    title="Divorces and Annulments (% of all marriages) before Xth Anniversary",
    xaxis_title="Marriage Cohort",
    yaxis_title="Divorces & Annulments (%)",
    showlegend=True,
    legend=dict(orientation="v", yanchor="top", y=0.99, xanchor="right", x=0.99),  # Place legend on the right of the graph
    barmode="stack",  # Use 'stack' for stacked cumulative bar chart
)

# Show the plot
fig.show()

In [168]:
# Create the column_names_mapping dictionary using dictionary comprehension
column_names_mapping = {col: snake_case_to_readable(col) for col in columns_to_plot[1:]}

# Create a figure
fig = go.Figure()

# Add trace for each selected column as a stacked cumulative bar chart
for column in columns_to_plot[1:]:
    if column != "marriage_cohort":
        fig.add_trace(go.Bar(
            x=df_incremental["marriage_cohort"],
            y=df_incremental[column],
            text=df_incremental[column],
            textposition='inside',
            insidetextanchor='middle',
            textangle=-90,
            name=column_names_mapping[column],  # Use the mapped name from the dictionary
        ))

# Update layout
fig.update_layout(
    title="Divorces and Annulments (% of all marriages) before Xth Anniversary",
    xaxis_title="Marriage Cohort",
    yaxis_title="Divorces & Annulments (%)",
    showlegend=True,
    legend=dict(orientation="v", yanchor="top", y=0.99, xanchor="right", x=0.99),  # Place legend on the right of the graph
    barmode="stack",  # Use 'stack' for stacked cumulative bar chart
)

# Show the plot
fig.show()
